# HW4 REPORT

`1811363_洪一帆`

## 修改内容说明

上一版上交的notebook包含了一些错误，之前之所以没有进行修正是为了将错误的结果展现出来。并且在[下文](##答案与实际之间的一些差异)中解释了具体原因。

这一版修正了该错误,并且获得了正确的结果。

## 实现内容
1. [索引构建](#索引构建与检索-(40%)) 使用BSBI方法模拟在内存不足的情况下的索引构建方式，并应用于布尔查询
2. [索引压缩](#索引压缩-(30%)) 使用可变长编码对构建的索引进行压缩
3. [布尔检索](#布尔联合检索-(10%)) 对空格分隔的单词查询进行联合（与）布尔检索
4. [注释的改进]() 尝试使用规范的类、函数注释，在预览的时候就能看到各变量类型就很开心
5. [gamma编码、gap编码的实现]()成功实现了gamma编码以进行索引的压缩

## 遇到的问题

>在第一版之所以没有修正是为了突出对于问题的分析过程
>
>we's在答案中是被认为一整个单词，但是使用nltk分词的时候是将其分成we ' s三个部分，因此在查询时是认为文档中有we这个单词，但是答案中没有，因此造成了这个差异


> 在编写merge函数的时候一直无法找到错误，最终发现在最前面的parse block就没有输出结果
>
>原因：相对路径编写错误，忘记加上根目录了
>```python
>td_pairs = self.parse_block(self.data_dir + '/' + block_dir_relative + '/')
>```
>
>此前为了debug方便，将notebook代码复制到py文件进行debug
>
>但是由于测试代码并未按照notebook，因此仅在自己的测试代码上跑通了，忽视了根目录。因此之后一直都没有能找到原因。
>（因为自己的测试代码明明是能跑通的就没把注意力放在那个上面。。。）

>还是一个parse block的问题
>
>258 = {str} 'cool\n'
>
>没有忽略文件末尾的\n导致cool无法和cool\n匹配，导致错误

## ps

下面一些注释掉的代码块均为继承的函数，已经将其整合到类中以方便进行debug

# HW4 布尔查询之BSBI与索引压缩

由于BSBI与索引压缩对于索引的存储方式有着一定的要求，所以之前的框架不再适用，本次作业使用斯坦福大学[CS 276 / LING 286: Information Retrieval and Web Search](https://web.stanford.edu/class/cs276/)课程的代码框架来实现。具体来说主要包含的内容有：
1. [索引构建 (40%)](#索引构建与检索-(40%)) 使用BSBI方法模拟在内存不足的情况下的索引构建方式，并应用于布尔查询
2. [索引压缩 (30%)](#索引压缩-(30%)) 使用可变长编码对构建的索引进行压缩
3. [布尔检索 (10%)](#布尔联合检索-(10%)) 对空格分隔的单词查询进行联合（与）布尔检索
3. [实验报告 (10%)](#Report-(25%)) 描述你的代码并回答一些问题
4. [额外的编码方式 (10%)](#额外的编码方式-(10%)) 鼓励使用额外的编码方式对索引进行压缩 (例如, gamma-encoding)

In [58]:
# You can add additional imports here
import sys
import pickle as pkl
import array
import os
import timeit
import contextlib
import zipfile
import nltk
from collections import defaultdict

# 数据集

实验使用的文本数据是stanford.edu域下的网页内容，可从http://web.stanford.edu/class/cs276/pa/pa1-data.zip 下载。以下代码将大约170MB的文本数据下载到当前目录下，

In [59]:
# import urllib.request
#
# data_url = 'http://web.stanford.edu/class/cs276/pa/pa1-data.zip'
# data_dir = 'pa1-data'
# urllib.request.urlretrieve(data_url, data_dir+'.zip')
# zip_ref = zipfile.ZipFile(data_dir+'.zip', 'r')
# zip_ref.extractall()
# zip_ref.close()

之后构建的索引会被存储到`output_dir`，`tmp`会存储测试数据（toy-data）所生成的一些临时文件

In [60]:
# try:
#     os.mkdir('output_dir')
# except FileExistsError:
#     pass
# try:
#     os.mkdir('tmp')
# except FileExistsError:
#     pass
# try:
#     os.mkdir('toy_output_dir')
# except FileExistsError:
#     pass

在数据目录下有10个子目录（命名0-9）

In [61]:
sorted(os.listdir('pa1-data'))

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

每一个子目录下的文件都包含一个独立网页的内容。可以认为在同一子目录下没有同名文件，即每个文件的绝对路径不会相同。

In [62]:
sorted(os.listdir('pa1-data/0'))[:10]

['3dradiology.stanford.edu_',
 '3dradiology.stanford.edu_patient_care_Case%2520studies_AVM.html',
 '3dradiology.stanford.edu_patient_care_case_studies.html',
 '5-sure.stanford.edu_',
 '50years.stanford.edu_',
 'a3cservices.stanford.edu_awards_nominate_',
 'a3cservices.stanford.edu_facilities_',
 'a3cservices.stanford.edu_lead_',
 'aa.stanford.edu_',
 'aa.stanford.edu_about_aviation.php']

所有的网页内容已经经过处理，仅包含由空格分隔开的单词，不再需要进行额外的标准化工作。

In [63]:
# with open('pa1-data.zip/pa1-data/0/3dradiology.stanford.edu_', 'r') as f:
#     print(f.read())

作业目录下有一个小的数据集文件夹`toy-data`。在使用完整网页数据集之前，我们会用它来测试我们的代码是否正确。

In [64]:
toy_dir = 'toy-data'

# 索引构建与检索 (40%)

作业的第一部分是使用**blocked sort-based indexing (BSBI)** 算法来构建倒排索引并实现布尔检索。关于BSBI算法可以参考老师课件或者斯坦福教材[Section 4.2](http://nlp.stanford.edu/IR-book/pdf/04const.pdf)。以下摘自教材内容

> To construct an index, we first make a pass through the collection assembling all term-docID pairs. We then sort the pairs with the term as the dominant key and docID as the secondary key. Finally, we organize the docIDs for each term into a postings list and compute statistics like term and document frequency. For small collections, all this can be done in memory. 

对于无法在内存一次性处理的较大数据集，将会使用到二级存储（如：磁盘）。

## IdMap

再次引用教材 Section 4.2:

> To make index construction more efficient, we represent terms as termIDs (instead of strings), where each termID is a unique serial number. We can build the mapping from terms to termIDs on the fly while we are processing the collection. Similarly, we also represent documents as docIDs (instead of strings).

我们首先定义一个辅助类`IdMap`，用于将字符串和数字ID进行相互映射，以满足我们在term和termID、doc和docID间转换的需求。

实现以下代码中的`_get_str` 和 `_get_id`函数，IdMap类的唯一接口是`__getitem__`，它是一个特殊函数，重写了下标运算`[]`,根据下标运算键的类型得到正确的映射值（如果不存在需要添加）。（特殊函数可参考[官方文档](https://docs.python.org/3.7/reference/datamodel.html#special-method-names)）
<br>
<br>
我们会用到字典来将字符串转换为数字，用列表来将数字转换为字符串。

In [65]:
class IdMap:
    """
        Helper class to store a mapping from strings to ids.

        :param self.id_to_str: list, 存储相应单词，其id为数组中的下标

    """

    def __init__(self):

        self.str_to_id = {}
        "self.id_to_str: list, 存储相应单词，其id为数组中的下标"
        self.id_to_str = []

    def __len__(self):
        """Return number of terms stored in the IdMap"""
        return len(self.id_to_str)

    def _get_str(self, i):
        """Returns the string corresponding to a given id (`i`)."""
        ### Begin your code
        if i >= self.__len__():
            raise IndexError
        return self.id_to_str[i]
        ### End your code

    def _get_id(self, s):
        """Returns the id corresponding to a string (`s`).
        If `s` is not in the IdMap yet, then assigns a new id and returns the new id.
        """
        ### Begin your code
        res = self.str_to_id.get(s)
        if res or res==0:
            return res
        return self._assign_id(s)
        ### End your code

    def _assign_id(self, s):
        """
        给一个不在id map中的单词分配id
        :param s: str
        :return: 分配的id
        """
        len = self.__len__()
        self.id_to_str.append(s)
        self.str_to_id[s] = len
        return len

    def __getitem__(self, key):
        """If `key` is a integer, use _get_str;
           If `key` is a string, use _get_id;"""
        if type(key) is int:
            return self._get_str(key)
        elif type(key) is str:
            return self._get_id(key)
        else:
            raise TypeError

确保代码能通过以下简单测试样例

In [66]:
testIdMap = IdMap()
assert testIdMap['a'] == 0, "Unable to add a new string to the IdMap"
assert testIdMap['bcd'] == 1, "Unable to add a new string to the IdMap"
# print(testIdMap['a'])
assert testIdMap['a'] == 0, "Unable to retrieve the id of an existing string"
assert testIdMap[1] == 'bcd', "Unable to retrive the string corresponding to a\
                                given id"
try:
    testIdMap[2]
except IndexError as e:
    assert True, "Doesn't throw an IndexError for out of range numeric ids"
assert len(testIdMap) == 2

之后会需要你自己来写测试样例来确保你的程序正常运行

## 将倒排列表编码成字节数组

为了高效地从磁盘读写倒排列表（文档ID），我们将其存储为字节数组的形式。代码提供了`UncompressedPostings`类来用静态函数实现对倒排列表的编码和解码。在之后的任务中你需要使用该接口实现索引压缩版本（可变长编码）。

参考:
1. https://docs.python.org/3/library/array.html
2. https://pymotw.com/3/array/#module-array

In [67]:
class UncompressedPostings:
    
    @staticmethod
    def encode(postings_list):
        """Encodes postings_list into a stream of bytes


        Parameters
        ----------
        postings_list: List[int]
            List of docIDs (postings)
            
        Returns
        -------
        bytes
            bytearray representing integers in the postings_list
        """
        return array.array('L', postings_list).tobytes()
        
    @staticmethod
    def decode(encoded_postings_list):
        """Decodes postings_list from a stream of bytes
        
        Parameters
        ----------
        encoded_postings_list: bytes
            bytearray representing encoded postings list as output by encode 
            function
            
        Returns
        -------
        List[int]
            Decoded list of docIDs from encoded_postings_list
        """
        
        decoded_postings_list = array.array('L')
        decoded_postings_list.frombytes(encoded_postings_list)
        return decoded_postings_list.tolist()

运行以下代码查看其工作方式

In [68]:
x = UncompressedPostings.encode([1,2,3])
print(x)
print(UncompressedPostings.decode(x))

b'\x01\x00\x00\x00\x02\x00\x00\x00\x03\x00\x00\x00'
[1, 2, 3]


## 磁盘上的倒排索引

> With main memory insufficient, we need to use an external sorting algorithm, that is, one that uses disk. For acceptable speed, the central requirement of such an algorithm is that it minimize the number of random disk seeks during sorting - sequential disk reads are far faster than seeks. 

在这一部分我们提供了一个基类`InvertedIndex`，之后会在此基础上构建它的子类`InvertedIndexWriter`, `InvertedIndexIterator` 和 `InvertedIndexMapper`。在Python中我们常用`cPickle`进行序列化，但是它并不支持部分读和部分写，无法满足BSBI算法的需要，所以我们需要定义自己的存储方式。

In [69]:
class InvertedIndex:
    """A class that implements efficient reads and writes of an inverted index
    to disk

    Attributes
    ----------
    postings_dict: Dictionary mapping: termID->(start_position_in_index_file,
                                                number_of_postings_in_list,
                                               length_in_bytes_of_postings_list)
        termID是输入
        This is a dictionary that maps from termIDs to a 3-tuple of metadata
        that is helpful in reading and writing the postings in the index file
        to/from disk. This mapping is supposed to be kept in memory.
        start_position_in_index_file is the position (in bytes) of the postings
        list in the index file
        number_of_postings_in_list is the number of postings (docIDs) in the
        postings list
        length_in_bytes_of_postings_list is the length of the byte
        encoding of the postings list

    terms: List[int]
        A list of termIDs to remember the order in which terms and their
        postings lists were added to index.

        After Python 3.7 we technically no longer need it because a Python dict
        is an OrderedDict, but since it is a relatively new feature, we still
        maintain backward compatibility with a list to keep track of order of
        insertion.



    """

    def __init__(self, index_name, postings_encoding=None, directory=''):
        """
        Parameters
        ----------
        index_name :(str) Name used to store files related to the index
        postings_encoding: A class implementing static methods for encoding and
            decoding lists of integers. Default is None, which gets replaced
            with UncompressedPostings
        directory :(str) Directory where the index files will be stored
        """

        self.index_file_path = os.path.join(directory, index_name + '.index')
        self.metadata_file_path = os.path.join(directory, index_name + '.dict')

        if postings_encoding is None:
            self.postings_encoding = UncompressedPostings
        else:
            self.postings_encoding = postings_encoding
        self.directory = directory

        self.postings_dict = defaultdict(list)  # 用于存储倒排索引term-postlist
        self.terms = []  # Need to keep track of the order in which the
        # terms were inserted. Would be unnecessary
        # from Python 3.7 onwards

    def __enter__(self):
        """Opens the index_file and loads metadata upon entering the context"""
        # Open the index file
        self.index_file = open(self.index_file_path, 'rb+')

        # Load the postings dict and terms from the metadata file
        with open(self.metadata_file_path, 'rb') as f:
            self.postings_dict, self.terms = pkl.load(f)
            self.term_iter = self.terms.__iter__()
            # print(self.postings_dict,self.terms)

        return self

    def __exit__(self, exception_type, exception_value, traceback):
        """Closes the index_file and saves metadata upon exiting the context"""
        # Close the index file
        self.index_file.close()

        # Write the postings dict and terms to the metadata file
        with open(self.metadata_file_path, 'wb') as f:
            pkl.dump([self.postings_dict, self.terms], f)

因为是在与磁盘上的文件进行交互，所以我们提供了`__enter__`和`__exit__`函数，它使得我们能够像使用python中文件IO一样使用`with`语句。（参考[上下文管理器官方文档](https://docs.python.org/3/library/contextlib.html)）

以下是使用 `InvertedIndexWriter` 上下文管理器的实例:

```python

with InvertedIndexWriter('test', directory='tmp/') as index:
    # Some code here
    index
```

## 索引

> BSBI
>(i) segments the collection into parts of equal size,
>(ii) sorts the termID-docID pairs of each part in memory,
>(iii) stores intermediate sorted results on disk, and
>(iv) merges all intermediate results into the final index.

你需要将每一个子目录当做一个块（block），并且在构建索引的过程中每次只能加载一个块（模拟内存不足）。注意到我们是将操作系统意义上的块进行了抽象。你可以认为每个块足够小，能被装载进内存。

在这一部分，我们将阶段性地构造类`BSBIIndex`。函数`index`给出了BSBI算法的框架，而你的工作则是在接下来的部分中实现函数`parse_block`, `invert_write` 和 `merge`。

In [70]:
# Do not make any changes here, they will be overwritten while grading
class BSBIIndex:
    """
    Attributes
    ----------
    term_id_map(IdMap): For mapping terms to termIDs
    doc_id_map(IdMap): For mapping relative paths of documents (eg
        0/3dradiology.stanford.edu_) to docIDs
    data_dir(str): Path to data
    output_dir(str): Path to output index files
    index_name(str): Name assigned to index
    postings_encoding: Encoding used for storing the postings.
        The default (None) implies UncompressedPostings
    """

    def __init__(self, data_dir, output_dir, index_name="BSBI",
                 postings_encoding=None):
        self.term_id_map = IdMap()
        self.doc_id_map = IdMap()
        self.data_dir = data_dir
        self.output_dir = output_dir
        self.index_name = index_name
        self.postings_encoding = postings_encoding

        # Stores names of intermediate indices
        self.intermediate_indices = []

    def save(self):
        """
        都是单独的map，而非倒排索引表
        Dumps doc_id_map and term_id_map into output directory"""

        with open(os.path.join(self.output_dir, 'terms.dict'), 'wb') as f:
            pkl.dump(self.term_id_map, f)
        with open(os.path.join(self.output_dir, 'docs.dict'), 'wb') as f:
            pkl.dump(self.doc_id_map, f)

    def load(self):
        """Loads doc_id_map and term_id_map from output directory"""

        with open(os.path.join(self.output_dir, 'terms.dict'), 'rb') as f:
            self.term_id_map = pkl.load(f)
        with open(os.path.join(self.output_dir, 'docs.dict'), 'rb') as f:
            self.doc_id_map = pkl.load(f)

    def merge(self, indices, merged_index):
        """
        未能实现完全的不利用多余内存的合并方法
        Merges multiple inverted indices into a single index
        利用merged index来进行写入磁盘



        import heapq
        animal_lifespans = [('Giraffe', 28),
                           ('Rhinoceros', 40),
                           ('Indian Elephant', 70),
                           ('Golden Eagle', 80),
                           ('Box turtle', 123)]

        tree_lifespans = [('Gray Birch', 50),
                          ('Black Willow', 70),
                          ('Basswood', 100),
                          ('Bald Cypress', 600)]

        lifespan_lists = [animal_lifespans, tree_lifespans]

        for merged_item in heapq.merge(*lifespan_lists, key=lambda x: x[1]):
            print(merged_item)


        Parameters
        ----------
        indices: List[InvertedIndexIterator]
            A list of InvertedIndexIterator objects, each representing an
            **iterable inverted index for a block**
        merged_index: InvertedIndexWriter
            An instance of InvertedIndexWriter object into which each merged
            postings list is written out one at a time
        """
        # print(len(self.term_id_map))
        # 此时我应该能够在一个循环中存下所有该词的posting list

        stack = defaultdict(list) # 用来保存next出来，但是不能立即加入到pl中的tuple元组
        for termid in range(len(self.term_id_map)):
            pl = []
            for iii in indices:
                try:
                    while iii:
                        tup = next(iii)
                        if tup[0] == termid:  # 完成一次插入就跳出，否则会直接遍历完iter的所有元素并且无法重新置位为起始
                            pl += tup[1]
                            break
                        stack[tup[0]]+=tup[1]  # 返回的tup[0]并非当前正在合并的term，因此将它放到列表中
                except StopIteration:
                    continue
            if stack.get(termid):
                pl+=stack.pop(termid)
            merged_index.append(termid, sorted(pl))

#         term_id = -1
#         postings_list = []
#         print('-' * 40)
#         print(self.term_id_map.id_to_str)
#         print(self.doc_id_map.id_to_str)
#         for pair in heapq.merge(*indices, key=lambda x: x[0]):
# #             print(pair)
#             if pair[0] != term_id:
#                 if term_id != -1:
#                     merged_index.append(term_id, sorted(set(postings_list)))
#                 term_id = pair[0]
#                 postings_list = []
#             postings_list += pair[1]
#         merged_index.append(term_id, sorted(set(postings_list)))

    def index(self):
        """Base indexing code

        既然我们知道文档列表已经排过序了，那么我们可以在线性时间内对它们进行合并排序。
        事实上heapq(参考文档) 是Python中一个实现了堆排序算法的标准模板。
        另外它还包含一个实用的函数heapq.merge，可以将多个已排好序的输入（可迭代）合并成一个有序的输出并返回它的迭代器。
        它不仅可以用来合并倒排列表，也可以合并倒排索引词典。

        This function loops through the data directories,
        calls parse_block to parse the documents
        calls invert_write, which inverts each block and writes to a new index
        then saves the id maps and calls merge on the intermediate indices
        """
        import time

        # 从顶层目录开始解析
        for block_dir_relative in sorted(next(os.walk(self.data_dir))[1]):
            time_start=time.time()

            td_pairs = self.parse_block(self.data_dir + '/' + block_dir_relative + '/')


            index_id = 'index_' + block_dir_relative
            self.intermediate_indices.append(index_id)
            # 将一个目录下的文件写入磁盘
            with InvertedIndexWriter(index_id, directory=self.output_dir,
                                     postings_encoding=
                                     self.postings_encoding) as index:
                self.invert_write(td_pairs, index)  # 应该没问题了
                td_pairs = None
            time_end=time.time()
            print("time=",time_end-time_start)
        self.save()
        with InvertedIndexWriter(self.index_name, directory=self.output_dir,
                                 postings_encoding=
                                 self.postings_encoding) as merged_index:
            # 入栈
            with contextlib.ExitStack() as stack:
                # 根据index id向stack压栈，组成一个列表，每个列表项indices[i]是一个
                indices = [stack.enter_context(
                    InvertedIndexIterator(index_id,
                                          directory=self.output_dir,
                                          postings_encoding=
                                          self.postings_encoding))
                    for index_id in self.intermediate_indices]
                # print(indices)
                time_start=time.time()

                self.merge(indices, merged_index)
                time_end=time.time()
                print("time=",time_end-time_start)

    def parse_block(self, block_dir_relative):
        """
        仅返回td pairs的list，非倒排索引表，未进行文档去重
        Parses a tokenized text file into termID-docID pairs

        Parameters
        ----------
        block_dir_relative : str
            Relative Path to the directory that contains the files for the block

        Returns
        -------

        List[(Int, Int)]
            Returns all the td_pairs extracted from the block
            # used to be List[Tuple[Int, Int]]

        Should use self.term_id_map and self.doc_id_map to get termIDs and docIDs.
        These persist across calls to parse_block
        """
        result = []
        for filepath, dirs, fs in os.walk(block_dir_relative):
            for filename in fs:
                fullpath = os.path.join(filepath, filename)
                with open(fullpath, 'r') as fr:
                    docID = self.doc_id_map.__getitem__(fullpath)
                    # pattern=re.compile('\S*?')
                    # words = pattern.findall(fr.read())
                    words=fr.read().strip().split(' ')
                    # print(words)
                    for word in words:
                        # 返回一个词的id
                        termID = self.term_id_map.__getitem__(word)
                        result.append((termID, docID))

        return result

    def invert_write(self, td_pairs, index):
        """
        写一个块的数据，index是一个块的writer
        构建倒排索引列表，并写入相应index文件
        Inverts td_pairs into postings_lists and writes them to the given index

        Parameters
        ----------
        td_pairs: List[Tuple[Int, Int]]
            一个块中的td pairs
            List of termID-docID pairs
            和parse blocks函数返回值相同
        index: InvertedIndexWriter
            对应块的index文件
            Inverted index on disk corresponding to the block
        """
        res = defaultdict(list)
        # 遍历所有td pairs并将其归类，形成term - doclist的inverted list
        for pair in sorted(td_pairs):
            termID = pair[0]
            docID = pair[1]
            res[termID].append(docID)
        terms = res.keys()
        # 将每个term 以及其postinglist写入磁盘
        for t in terms:
            index.append(t, sorted((set(res[t]))))
            
            
    def retrieve(self, query):
        """Retrieves the documents corresponding to the conjunctive query

        Parameters
        ----------
        query: str
            Space separated list of query tokens

        Result
        ------
        List[str]
            Sorted list of documents which contains each of the query tokens.
            Should be empty if no documents are found.

        Should NOT throw errors for terms not in corpus
        """
        if len(self.term_id_map) == 0 or len(self.doc_id_map) == 0:
            self.load()

        ### Begin your code
        terms = query.split(' ')
        with InvertedIndexMapper(self.index_name, self.postings_encoding, directory=self.output_dir) as iim:
            res = []

            for term in terms:
                try:
                    termid=self.term_id_map.str_to_id[term]
                    if not res:
                        res = iim._get_postings_list(termid)
                    res = sorted_intersect(res, iim._get_postings_list(termid))
                except Exception:
                    print(term, '没找到')
                    continue
            return [self.doc_id_map.id_to_str[r] for r in res]
        ### End your code

# BSBI_instance = BSBIIndex(data_dir='test-pa1-data1', output_dir = 'test-output-diroutput_dir' )
# BSBI_instance.index()

### 解析

> The function `parse_block`  parses documents into termID-docID pairs and accumulates the pairs in memory until a block of a fixed size is full. We choose the block size to fit comfortably into memory to permit a fast in-memory sort. 

你需要将每一个子目录当做一个块，`parse_block`接收子目录路径作为参数。同一子目录下所有文件名都是不同的。

_注意 - 我们使用 `BSBIIndex` 继承 `BSBIIndex`, 这只是对一个已存在类添加新内容的简单方法。在这里只是用来切分类的定义（jupyter notebook内教学使用，无特殊含义）。_

In [71]:
# class BSBIIndex(BSBIIndex):
#     def parse_block(self, block_dir_relative):
#         """
#         仅返回td pairs的list，非倒排索引表，未进行文档去重
#         Parses a tokenized text file into termID-docID pairs
# 
#         Parameters
#         ----------
#         block_dir_relative : str
#             Relative Path to the directory that contains the files for the block
# 
#         Returns
#         -------
# 
#         List[(Int, Int)]
#             Returns all the td_pairs extracted from the block
#             # used to be List[Tuple[Int, Int]]
# 
#         Should use self.term_id_map and self.doc_id_map to get termIDs and docIDs.
#         These persist across calls to parse_block
#         """
#         result = []
#         for filepath, dirs, fs in os.walk(block_dir_relative):
#             for filename in fs:
#                 fullpath = os.path.join(filepath, filename)
#                 with open(fullpath, 'r') as fr:
#                     docID = self.doc_id_map.__getitem__(fullpath)
#                     words = nltk.word_tokenize(fr.read())
#                     for word in words:
#                         # 返回一个词的id
#                         termID = self.term_id_map.__getitem__(word)
#                         result.append((termID, docID))
# 
#         return sorted(result)

观察函数在测试数据上是否正常运行

In [72]:
with open('toy-data/0/fine.txt', 'r') as f:
    print(f.read())
with open('toy-data/0/hello.txt', 'r') as f:
    print(f.read())

i'm fine , thank you

hi hi
how are you ?



写一些测试样例来确保`parse_block`方法正常运行（如：相同单词出现时是相同ID）

In [73]:
### Begin your code
BSBI_instance = BSBIIndex(data_dir=toy_dir, output_dir = 'tmp/', index_name = 'toy')
BSBI_instance.parse_block('toy-data/1/')
### End your code

[(0, 0),
 (1, 0),
 (2, 0),
 (3, 0),
 (4, 0),
 (5, 0),
 (3, 0),
 (6, 0),
 (7, 0),
 (8, 1)]

### 倒排表

> The block is then inverted and written to disk. Inversion involves two steps. First, we sort the termID-docID pairs. Next, we collect all termID-docID pairs with the same termID into a postings list, where a posting is simply a docID. The result, an inverted index for the block we have just read, is then written to disk.

在这一部分我们添加函数`invert_write`来实现由termID-docID对构建倒排表。 

但是，我们首先需要实现类`InvertedIndexWriter`。和列表类似，该类提供了append函数，但是倒排表不会存储在内存中而是直接写入到磁盘里。

In [74]:
class InvertedIndexWriter(InvertedIndex):
    """

    Attributes
    ----------
    terms: list[int]
        term ids
    postings_dict: Dictionary mapping: termID->(start_position_in_index_file,
                                            number_of_postings_in_list,
                                           length_in_bytes_of_postings_list)
    index_file: context manager
        write metadata index
    append(term, postings_list):
        将postinglist的信息记录到posting dict
        注意：直接写入磁盘！！！
    """

    def __enter__(self):
        self.index_file = open(self.index_file_path, 'wb+')
        return self

    def append(self, term, postings_list):
        """
        需要一次性构建完整的posting list吗？？？？
        Appends the term and encoded postings_list to end of the index file.
        并且将postinglist的信息记录到posting dict
        注意：直接写入磁盘！！！



        This function does three things,
        1. Encodes the postings_list using self.postings_encoding
        2. Stores metadata in the form of self.terms and self.postings_dict
           Note that self.postings_dict maps termID to a 3 tuple of
           (start_position_in_index_file,
           number_of_postings_in_list,
           length_in_bytes_of_postings_list)
        3. Appends the bytestream to the index file on disk

        Hint: You might find it helpful to read the Python I/O docs
        (https://docs.python.org/3/tutorial/inputoutput.html) for
        information about appending to the end of a file.

        Parameters
        ----------
        term:
            term or termID is the unique identifier for the term
            最好是termID吧
        postings_list: List[Int]
            List of docIDs where the term appears


        """
        # print(term,postings_list)
        encoded_postlist = self.postings_encoding.encode(postings_list)
        # if term not in self.terms:
        self.terms.append(term)
        # 不会覆盖，因为term是独立的。
        # self.index_file.tell()用于指明当前indexfile的末端位置
        self.postings_dict[term] = (self.index_file.tell(), len(postings_list), len(encoded_postlist))
        self.index_file.write(encoded_postlist)  # self.index_file可以被写入数据




尽管还没有实现读取索引的类，我们还是可以用以下测试代码检测我们的实现。

In [75]:
with InvertedIndexWriter('test', directory='tmp/') as index:
    index.append(1, [2, 3, 4])
    index.append(2, [3, 4, 5])
    index.index_file.seek(0)
    assert index.terms == [1,2], "terms sequence incorrect"
    assert index.postings_dict == {1: (0, 3, len(UncompressedPostings.encode([2,3,4]))), 
                                   2: (len(UncompressedPostings.encode([2,3,4])), 3, 
                                       len(UncompressedPostings.encode([3,4,5])))}, "postings_dict incorrect"
    assert UncompressedPostings.decode(index.index_file.read()) == [2, 3, 4, 3, 4, 5], "postings on disk incorrect"

现在我们实现 `invert_write`，它将解析得到的td_pairs转换成倒排表，并使用`InvertedIndexWriter` 类将其写入磁盘。

In [76]:
# class BSBIIndex(BSBIIndex):
#     def invert_write(self, td_pairs, index):
#         """
#         写一个块的数据，index是一个块的writer
#         构建倒排索引列表，并写入相应index文件
#         Inverts td_pairs into postings_lists and writes them to the given index
# 
#         Parameters
#         ----------
#         td_pairs: List[Tuple[Int, Int]]
#             一个块中的td pairs
#             List of termID-docID pairs
#             和parse blocks函数返回值相同
#         index: InvertedIndexWriter
#             对应块的index文件
#             Inverted index on disk corresponding to the block
#         """
#         res = defaultdict(list)
#         # 遍历所有td pairs并将其归类，形成term - doclist的inverted list
#         for pair in td_pairs:
#             termID = pair[0]
#             docID = pair[1]
#             res[termID].append(docID)
#         terms = sorted(res.keys())
#         # 将每个term 以及其postinglist写入磁盘
#         for t in terms:
#             # print("inverted")
#             index.append(t, sorted(list(set(res[t]))))
#             # print(list(set(res[t])))
#         


我们可以在测试数据上读取一个块并观察倒排索引中包含的内容。
仿照`InvertedIndexWriter`部分写一些测试样例。

In [77]:
### Begin your code

# bsbi.load()
# print(bsbi.parse_block('toy-data'))
with InvertedIndexWriter('test', directory=r"tmp/") as iiw :
    bsbi = BSBIIndex('toy-data', 'tmp', 'test')
    bsbi.invert_write(bsbi.parse_block('toy-data'), iiw)
### End your code

### 合并
> The algorithm simultaneously merges the ten blocks into one large merged index. To do the merging, we open all block files simultaneously, and maintain small read buffers for the ten blocks we are reading and a write buffer for the final merged index we are writing. 

Python中的迭代模型非常自然地符合我们维护一个小的读缓存的要求。我们可以迭代地从磁盘上每次读取文件的一个倒排列表。我们通过构建`InvertedIndex`的子类`InvertedIndexIterator`来完成这个迭代任务。

In [78]:
class InvertedIndexIterator(InvertedIndex):
    """
    用next函数逐个获取term和posting list

    file.read([size])：size 未指定则返回整个文件，如果文件大小 >2 倍内存则有问题，f.read()读到文件尾时返回""(空字串)。

    f.tell()：返回一个整数,表示当前文件指针的位置(就是到文件头的字节数)。

    f.seek(偏移量,[起始位置])：用来移动文件指针。

    偏移量: 单位为字节，可正可负
    起始位置: 0 - 文件头, 默认值; 1 - 当前位置; 2 - 文件尾
    f.close() 关闭文件

    Attributes
    ----------
    indicator: int
        用于指明当前遍历的字节流index file的位置


    """

    def __enter__(self):
        """Adds an initialization_hook to the __enter__ function of super class
        """
        super().__enter__()
        self._initialization_hook()
        return self

    def _initialization_hook(self):
        """
        初始化一个标志位，用于标明当前遍历的字节流index file的位置
        """
        ### Begin your code
        # self.indicator=0
        ### End your code

    def __iter__(self):
        return self

    def __next__(self):
        """
        Returns the next (term, postings_list) pair in the index.

        file.read([size])：size 未指定则返回整个文件，如果文件大小 >2 倍内存则有问题，f.read()读到文件尾时返回""(空字串)。

        f.tell()：返回一个整数,表示当前文件指针的位置(就是到文件头的字节数)。

        f.seek(偏移量,[起始位置])：用来移动文件指针。

        偏移量: 单位为字节，可正可负
        起始位置: 0 - 文件头, 默认值; 1 - 当前位置; 2 - 文件尾

        Note: This function should only read a small amount of data from the
        index file. In particular, you should not try to maintain the full
        index file in memory.

        Returns
        ---------
        term: int
            单词的编号 term id
        posting_list: list[int]
            倒排索引表

        """
        ### Begin your code
        term = next(self.term_iter)
        pd = self.postings_dict[term]
        self.index_file.seek(pd[0])  # 将文件指针向后移，read时以文件指针位置为基址
        encoded_posting_list = self.index_file.read(pd[2])
        return term, self.postings_encoding.decode(encoded_posting_list)
        ### End your code

    def delete_from_disk(self):
        """Marks the index for deletion upon exit. Useful for temporary indices
        """
        self.delete_upon_exit = True

    def __exit__(self, exception_type, exception_value, traceback):
        """Delete the index file upon exiting the context along with the
        functions of the super class __exit__ function"""
        self.index_file.close()
        if hasattr(self, 'delete_upon_exit') and self.delete_upon_exit:
            os.remove(self.index_file_path)
            os.remove(self.metadata_file_path)
        else:
            with open(self.metadata_file_path, 'wb') as f:
                pkl.dump([self.postings_dict, self.terms], f)

为了测试以上代码，我们先用`InvertedIndexWriter` 创建索引，然后再迭代遍历。写一些小的测试样例观察它是否正常运行。至少你得写一个测试，手工构建一个小的索引，用`InvertedIndexWriter`将他们写入磁盘，然后用`InvertedIndexIterator`迭代遍历倒排索引。

In [79]:
"""test sample"""
### Begin your code
iii = InvertedIndexIterator('test', directory=r"tmp")
with iii as i:#这些iter、next函数只有在被当做上下文管理器打开时才能访问到
    t = iter(i)
    try:
        while t:
            print(t.__next__())
    except StopIteration:
        i.__exit__(StopIteration,StopIteration.value,StopIteration.__traceback__)
### End your code

(0, [0, 5, 7])
(1, [0, 5, 7])
(2, [0, 3, 5, 7])
(3, [0, 5, 7])
(4, [0, 1, 2, 3, 5, 6, 7, 8])
(5, [1, 6, 8])
(6, [1, 6, 8])
(7, [1, 2, 6, 8])
(8, [1, 3, 6, 8])
(9, [2])
(10, [2])
(11, [2])
(12, [2])
(13, [2])
(14, [2])
(15, [2])
(16, [2])
(17, [2])
(18, [2])
(19, [2])
(20, [2])
(21, [2])
(22, [2])
(23, [2])
(24, [2])
(25, [2])
(26, [2, 3])
(27, [2])
(28, [2])
(29, [2])
(30, [2])
(31, [2])
(32, [2])
(33, [2])
(34, [2])
(35, [2])
(36, [2])
(37, [2])
(38, [2])
(39, [2])
(40, [2])
(41, [2])
(42, [2])
(43, [2])
(44, [2])
(45, [2])
(46, [2])
(47, [2])
(48, [2])
(49, [2])
(50, [2])
(51, [2])
(52, [2])
(53, [2])
(54, [2])
(55, [2])
(56, [2])
(57, [2])
(58, [2])
(59, [2])
(60, [2])
(61, [2])
(62, [2])
(63, [2])
(64, [2])
(65, [2])
(66, [2])
(67, [2])
(68, [2])
(69, [2])
(70, [2])
(71, [2])
(72, [2])
(73, [2])
(74, [2])
(75, [2])
(76, [2])
(77, [2])
(78, [2])
(79, [2])
(80, [2])
(81, [2])
(82, [2])
(83, [2])
(84, [2])
(85, [2])
(86, [2])
(87, [2])
(88, [2])
(89, [2])
(90, [2])
(91, [2])
(92, [2])

> During merging, in each iteration, we select the lowest termID that has not been processed yet using a priority queue or a similar data structure. All postings lists for this termID are read and merged, and the merged list is written back to disk. Each read buffer is refilled from its file when necessary.

我们将使用`InvertedIndexIterator`来完成读取的部分，用`InvertedIndexWriter`来将合并后的索引写入磁盘。

注意到我们之前一直使用`with` 语句来打开倒排索引文件，但是现在需要程序化地完成这项工作。可以看到我们在基类`BSBIIndex`的`index`函数中使用了`contextlib`([参考文档](https://docs.python.org/3/library/contextlib.html#contextlib.ExitStack))
你的任务是合并*打开的*`InvertedIndexIterator`对象（倒排列表），并且通过一个`InvertedIndexWriter`对象每次写入一个文档列表。

既然我们知道文档列表已经排过序了，那么我们可以在线性时间内对它们进行合并排序。事实上`heapq`([参考文档](https://docs.python.org/3.0/library/heapq.html)) 是Python中一个实现了堆排序算法的标准模板。另外它还包含一个实用的函数`heapq.merge`，可以将多个已排好序的输入（可迭代）合并成一个有序的输出并返回它的迭代器。它不仅可以用来合并倒排列表，也可以合并倒排索引词典。

为了让你快速上手`heapq.merge`函数，我们提供了一个简单的使用样例。给出两个以动物和树平均寿命排序的列表，我们希望合并它们。

In [80]:
import heapq
animal_lifespans = [('Giraffe', 28), 
                   ('Rhinoceros', 40), 
                   ('Indian Elephant', 70), 
                   ('Golden Eagle', 80), 
                   ('Box turtle', 123)]

tree_lifespans = [('Gray Birch', 50), 
                  ('Black Willow', 70), 
                  ('Basswood', 100),
                  ('Bald Cypress', 600)]

lifespan_lists = [animal_lifespans, tree_lifespans]

for merged_item in heapq.merge(*lifespan_lists, key=lambda x: x[1]):
    print(merged_item)

('Giraffe', 28)
('Rhinoceros', 40)
('Gray Birch', 50)
('Indian Elephant', 70)
('Black Willow', 70)
('Golden Eagle', 80)
('Basswood', 100)
('Box turtle', 123)
('Bald Cypress', 600)


注意使用`*`将`lifespan_lists`解包作为参数，并且使用`lambda`函数来给出进行排序的key。如果你对它们不熟悉可以参考文档([unpacking lists](https://docs.python.org/3/tutorial/controlflow.html#unpacking-argument-lists), [lambda expressions](https://docs.python.org/3/tutorial/controlflow.html#lambda-expressions))。

In [81]:
# import heapq
# class BSBIIndex(BSBIIndex):
#     def merge(self, indices, merged_index):
#         """
#         未能实现完全的不利用多余内存的合并方法
#         Merges multiple inverted indices into a single index
#         利用merged index来进行写入磁盘
# 
# 
# 
#         import heapq
#         animal_lifespans = [('Giraffe', 28),
#                            ('Rhinoceros', 40),
#                            ('Indian Elephant', 70),
#                            ('Golden Eagle', 80),
#                            ('Box turtle', 123)]
# 
#         tree_lifespans = [('Gray Birch', 50),
#                           ('Black Willow', 70),
#                           ('Basswood', 100),
#                           ('Bald Cypress', 600)]
# 
#         lifespan_lists = [animal_lifespans, tree_lifespans]
# 
#         for merged_item in heapq.merge(*lifespan_lists, key=lambda x: x[1]):
#             print(merged_item)
# 
# 
#         Parameters
#         ----------
#         indices: List[InvertedIndexIterator]
#             A list of InvertedIndexIterator objects, each representing an
#             **iterable inverted index for a block**
#         merged_index: InvertedIndexWriter
#             An instance of InvertedIndexWriter object into which each merged
#             postings list is written out one at a time
#         """
#         # print(len(self.term_id_map))
#         # 此时我应该能够在一个循环中存下所有该词的posting list
#         stack = defaultdict(list) # 用来保存next出来，但是不能立即加入到pl中的tuple元组
#         for termid in range(len(self.term_id_map)):
#             pl = []
#             for iii in indices:
#                 try:
#                     while iii:
#                         tup = next(iii)
#                         if tup[0] == termid:  # 完成一次插入就跳出，否则会直接遍历完iter的所有元素并且无法重新置位为起始
#                             pl += tup[1]
#                             break
#                         stack[tup[0]]+=tup[1]  # 返回的tup[0]并非当前正在合并的term，因此将它放到列表中
#                 except StopIteration:
#                     continue
#             if stack.get(termid):
#                 pl+=stack.pop(termid)
#             merged_index.append(termid, sorted(pl))

首先确保它在测试数据上正常运行

In [82]:
# BSBI_instance = BSBIIndex(data_dir=toy_dir, output_dir = 'toy_output_dir')
# BSBI_instance.index()

接下来对整个数据集构建索引

In [83]:
# BSBI_instance = BSBIIndex(data_dir='pa1-data', output_dir = 'output_dir')
# BSBI_instance.index()

如果你在合并阶段出现了错误，可以利用以下代码来debug。

In [84]:
# BSBI_instance = BSBIIndex(data_dir='pa1-data', output_dir = 'output_dir')
# BSBI_instance.intermediate_indices = ['index_'+str(i) for i in range(1)]
# with InvertedIndexWriter(BSBI_instance.index_name, directory=BSBI_instance.output_dir, postings_encoding=BSBI_instance.postings_encoding) as merged_index:
#     with contextlib.ExitStack() as stack:
#         indices = [stack.enter_context(InvertedIndexIterator(index_id, directory=BSBI_instance.output_dir, postings_encoding=BSBI_instance.postings_encoding)) for index_id in BSBI_instance.intermediate_indices]
#         BSBI_instance.merge(indices, merged_index)

## 布尔联合检索 (10%)

我们将实现BSBIIndex中的`retrieve`函数，对于给定的包含由空格分隔tokens的字符串查询，返回包含查询中所有tokens的文档列表。但是我们并不能迭代遍历整个索引或者将整个索引加载到内存来寻找相应的terms（索引太大）。

首先我们要实现`InvertedIndex`的子类`InvertedIndexMapper`，它能够找到对应terms在索引文件中位置并取出它的倒排记录表。

In [85]:
class InvertedIndexMapper(InvertedIndex):
    def __getitem__(self, key):
        return self._get_postings_list(key)
    
    def _get_postings_list(self, term):
        """Gets a postings list (of docIds) for `term`.
        
        This function should not iterate through the index file.
        I.e., it should only have to read the bytes from the index file
        corresponding to the postings list for the requested term.
        """
        ### Begin your code
        try:
            pd = self.postings_dict[term]
            if not pd:
                raise KeyError
            self.index_file.seek(pd[0])  # 将文件指针向后移，read时以文件指针位置为基址
            encoded_posting_list = self.index_file.read(pd[2])
            return self.postings_encoding.decode(encoded_posting_list)
        except KeyError:
            print(term,'not found')


        ### End your code

写一些测试样例检测`_get_postings_list`的实现

In [86]:
### Begin your code
BSBI_instance = BSBIIndex(data_dir='test-pa1-data1', output_dir = 'test-output-diroutput_dir' )
# BSBI_instance.index()
with InvertedIndexMapper(BSBI_instance.index_name,BSBI_instance.postings_encoding,directory=BSBI_instance.output_dir) as iim:
    iim._get_postings_list("boolean")
### End your code

boolean not found


现在我们获得了查询中terms对应的倒排记录表，接着需要求他们的交集。这部分与我们之前作业的merge方法类似，请实现`sorted_intersect`函数，遍历两个有序列表并在线性时间内合并。

In [87]:
def sorted_intersect(list1, list2):
    """Intersects two (ascending) sorted lists and returns the sorted result
    
    Parameters
    ----------
    list1: List[Comparable]
    list2: List[Comparable]
        Sorted lists to be intersected
        
    Returns
    -------
    List[Comparable]
        Sorted intersection        
    """
    ### Begin your code
    return sorted(list(set(list1).intersection(set(list2))))
    ### End your code

简单的测试样例

In [88]:
### Begin your code
sorted_intersect([2,3,4,1],[3,4,2,1])
### End your code

[1, 2, 3, 4]

现在可以利用`sorted_intersect` 和 `InvertedIndexMapper`来实现`retrieve`函数。

In [89]:
# # %%tee submission/retrieve.py
# class BSBIIndex(BSBIIndex):
#     def retrieve(self, query):
#         """Retrieves the documents corresponding to the conjunctive query
# 
#         Parameters
#         ----------
#         query: str
#             Space separated list of query tokens
# 
#         Result
#         ------
#         List[str]
#             Sorted list of documents which contains each of the query tokens.
#             Should be empty if no documents are found.
# 
#         Should NOT throw errors for terms not in corpus
#         """
#         if len(self.term_id_map) == 0 or len(self.doc_id_map) == 0:
#             self.load()
# 
#         ### Begin your code
#         terms = query.split(' ')
#         with InvertedIndexMapper(self.index_name, self.postings_encoding, directory=self.output_dir) as iim:
#             res = []
# 
#             for term in terms:
#                 try:
#                     termid=self.term_id_map.str_to_id[term]
#                     if not res:
#                         res = iim._get_postings_list(termid)
#                     res = sorted_intersect(res, iim._get_postings_list(termid))
#                 except Exception:
#                     print(term, '没找到')
#                     continue
#             return [self.doc_id_map.id_to_str[r] for r in res]
#         ### End your code

通过一个简单的查询来测试索引在真实数据集上是否正常工作。

In [90]:
BSBI_instance = BSBIIndex(data_dir='pa1-data', output_dir = 'output_dir')
BSBI_instance.retrieve('boolean retrieval')

['pa1-data/1/cs276.stanford.edu_',
 'pa1-data/1/cs276a.stanford.edu_',
 'pa1-data/3/infolab.stanford.edu_TR_CS-TN-95-21.html',
 'pa1-data/4/nlp.stanford.edu_IR-book_',
 'pa1-data/4/nlp.stanford.edu_IR-book_html_htmledition_an-appraisal-of-probabilistic-models-1.html',
 'pa1-data/4/nlp.stanford.edu_IR-book_html_htmledition_bayesian-network-approaches-to-ir-1.html',
 'pa1-data/4/nlp.stanford.edu_IR-book_html_htmledition_boolean-retrieval-2.html',
 'pa1-data/4/nlp.stanford.edu_IR-book_html_htmledition_computing-scores-in-a-complete-search-system-1.html',
 'pa1-data/4/nlp.stanford.edu_IR-book_html_htmledition_dictionaries-and-tolerant-retrieval-1.html',
 'pa1-data/4/nlp.stanford.edu_IR-book_html_htmledition_efficient-scoring-and-ranking-1.html',
 'pa1-data/4/nlp.stanford.edu_IR-book_html_htmledition_inexact-top-k-document-retrieval-1.html',
 'pa1-data/4/nlp.stanford.edu_IR-book_html_htmledition_probabilistic-information-retrieval-1.html',
 'pa1-data/4/nlp.stanford.edu_IR-book_html_htmledit

你也可以通过读取文件来测试其中的页面是否真的包含了查询的terms

In [91]:
with open("pa1-data/1/cs276.stanford.edu_", 'r') as f:
    print(f.read())

cs276 information retrieval and web search cs 276 ling 286 information retrieval and web search spring 2011 pandu nayak and prabhakar raghavan lecture 3 units tu th 4 15 5 30 gates b03 tas sonali aggarwal ivan cui valentin spitkovsky and sandeep sripada staff e mail cs276 spr1011 staff lists stanford edu lectures are also available online and on television through scpd sitn course description basic and advanced techniques for text based information systems efficient text indexing boolean and vector space retrieval models evaluation and interface issues web search including crawling link based algorithms and web metadata text web clustering classification text mining syllabus additional information staff contact information piazzza we strongly recommend students to post questions to the course page on www piazzza com instead of sending emails this forum enables students to discuss the questions they encounter in lectures or assignments here is a quick introduction video email if you hav

测试dev queries（提前构建好的查询与结果）是否正确

In [92]:
BSBI_instance = BSBIIndex(data_dir='pa1-data', output_dir = 'output_dir')
for i in range(1, 9):
    with open('dev_queries/query.' + str(i)) as q:
        query = q.read().strip('\n')
        my_results = sorted(set([os.path.normpath(path) for path in BSBI_instance.retrieve(query)]))
        with open('dev_output/' + str(i) + '.out') as o:
            reference_results = sorted(set(['pa1-data\\' + os.path.normpath(x.strip()) for x in o.readlines()]))
            try:
                assert my_results==reference_results
                print("Results match for query:", query.strip())
            except AssertionError:
                print(set(my_results) - set(reference_results))
                print(set(reference_results) - set(my_results))
                print(str(i) + " Results DO NOT match for query: " + query.strip())

Results match for query: we are
Results match for query: stanford class
Results match for query: stanford students
Results match for query: very cool
Results match for query: the
Results match for query: a
Results match for query: the the
Results match for query: stanford computer science


如果出现了错误，可以通过以下代码比较输出与正确结果的差异

### 发现原因
we's在答案中是被认为一整个单词，但是使用nltk分词的时候是将其分成we ' s三个部分，因此在查询时是认为文档中有we这个单词，但是答案中没有，因此造成了这个差异

In [93]:
set(my_results) - set(reference_results)

set()

In [94]:
set(reference_results) - set(my_results)

set()

确保你构建自己的查询来测试所有可能的边界情况，例如数据集中没有出现的terms，或者拖慢合并的频繁出现的terms。

# 索引压缩  (30%)

在这部分，你将使用可变长字节编码对索引进行压缩。（gap-encoding可选，对gap进行编码）

下面几个Python运算符可能对你有帮助

In [95]:
# Remainder (modulo) operator %

print("10 % 2 = ", 10 % 2)
print("10 % 3 = ", 10 % 3)

# Integer division in Python 3 is done by using two slash signs

print("10 / 3 = ", 10 / 3)
print("10 // 3 = ", 10 // 3)

10 % 2 =  0
10 % 3 =  1
10 / 3 =  3.3333333333333335
10 // 3 =  3


完成下面的`CompressedPostings`类，我们将用它来替换`UncompressedPostings`。关于如何使用gap-encoding和可变长字节编码，你可以参考老师课件或者[Chapter 5](https://nlp.stanford.edu/IR-book/pdf/05comp.pdf)。

In [96]:
import array
class CompressedPostings:
    # If you need any extra helper methods you can add them here
    ### Begin your code

    ### End your code

    @staticmethod
    def encode(postings_list):
        """Encodes `postings_list` using gap encoding with variable byte
        encoding for each gap

        Parameters
        ----------
        postings_list: List[int]
            The postings list to be encoded

        Returns
        -------
        bytes:
            Bytes reprsentation of the compressed postings list
            (as produced by `array.tobytes` function)
        """
        ### Begin your code
        miner=[0]+postings_list[0:-1]
        postings_list=list(map(lambda x:x[0]-x[1],zip(postings_list,miner)))
        return array.array('L', postings_list).tobytes()

        ### End your code

    @staticmethod
    def decode(encoded_postings_list):
        """Decodes a byte representation of compressed postings list

        Parameters
        ----------
        encoded_postings_list: bytes
            Bytes representation as produced by `CompressedPostings.encode`

        Returns
        -------
        List[int]
            Decoded postings list (each posting is a docIds)
        """
        ### Begin your code
        base=0
        decoded_postings_list = array.array('L')
        decoded_postings_list.frombytes(encoded_postings_list)
        postings_list=decoded_postings_list.tolist()
        for i in range(len(postings_list)):
            base+=postings_list[i]
            postings_list[i]=base
        return  postings_list

        ### End your code


我们实现了一个简单的函数来测试你编码的列表是否被正确解码

In [97]:
def test_encode_decode(l):
    e = CompressedPostings.encode(l)
    d = CompressedPostings.decode(e)
    assert d == l,'wrong'
    print(d, e)

写一些测试样例来确保文档列表的压缩和解压正常运行

In [98]:
### Begin your code
test_encode_decode([1,2,3,4])
### End your code

[1, 2, 3, 4] b'\x01\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00'


现在让我们创建一个新的输出文件夹并使用`CompressedPostings`来构建压缩索引

In [99]:
try: 
    os.mkdir('output_dir_compressed')
except FileExistsError:
    pass

In [100]:
#test
BSBI_instance_compressed = BSBIIndex(data_dir='test-pa1-data1', output_dir = 'test_output_dir_compressed', postings_encoding=CompressedPostings)
# BSBI_instance_compressed.index()

In [101]:
BSBI_instance_compressed = BSBIIndex(data_dir='pa1-data', output_dir = 'output_dir_compressed', postings_encoding=CompressedPostings)
# BSBI_instance_compressed.index()

In [102]:
BSBI_instance_compressed.retrieve('boolean retrieval')

['pa1-data/1/cs276.stanford.edu_',
 'pa1-data/1/cs276a.stanford.edu_',
 'pa1-data/3/infolab.stanford.edu_TR_CS-TN-95-21.html',
 'pa1-data/4/nlp.stanford.edu_IR-book_',
 'pa1-data/4/nlp.stanford.edu_IR-book_html_htmledition_an-appraisal-of-probabilistic-models-1.html',
 'pa1-data/4/nlp.stanford.edu_IR-book_html_htmledition_bayesian-network-approaches-to-ir-1.html',
 'pa1-data/4/nlp.stanford.edu_IR-book_html_htmledition_boolean-retrieval-2.html',
 'pa1-data/4/nlp.stanford.edu_IR-book_html_htmledition_computing-scores-in-a-complete-search-system-1.html',
 'pa1-data/4/nlp.stanford.edu_IR-book_html_htmledition_dictionaries-and-tolerant-retrieval-1.html',
 'pa1-data/4/nlp.stanford.edu_IR-book_html_htmledition_efficient-scoring-and-ranking-1.html',
 'pa1-data/4/nlp.stanford.edu_IR-book_html_htmledition_inexact-top-k-document-retrieval-1.html',
 'pa1-data/4/nlp.stanford.edu_IR-book_html_htmledition_probabilistic-information-retrieval-1.html',
 'pa1-data/4/nlp.stanford.edu_IR-book_html_htmledit

像之前一样，用已构造好的查询语句来测试是否正常运行

In [103]:
BSBI_instance = BSBIIndex(data_dir='pa1-data', output_dir='output_dir_compressed',postings_encoding=CompressedPostings)
for i in range(1, 9):
    with open('dev_queries/query.' + str(i)) as q:
        query = q.read().strip('\n')
        my_results = sorted(set([os.path.normpath(path) for path in BSBI_instance.retrieve(query)]))
        with open('dev_output/' + str(i) + '.out') as o:
            reference_results = sorted(set(['pa1-data\\' + os.path.normpath(x.strip()) for x in o.readlines()]))
            try:
                assert my_results==reference_results
                print("Results match for query:", query.strip())
            except AssertionError:
                print(set(my_results) - set(reference_results))
                print(set(reference_results) - set(my_results))
                print(str(i) + " Results DO NOT match for query: " + query.strip())

Results match for query: we are
Results match for query: stanford class
Results match for query: stanford students
Results match for query: very cool
Results match for query: the
Results match for query: a
Results match for query: the the
Results match for query: stanford computer science


In [104]:
set(my_results) - set(reference_results)

set()

In [105]:
set(reference_results) - set(my_results)

set()

# 额外的编码方式 (10%)

通过补充`ECCompressedPostings`的`encode` 和 `decode`方法来实现一种额外的索引压缩方式。在我们课上学的就是**gamma-encoding** 。另外如果大家感兴趣的话也可以了解**Delta Encoding** ，[ALGORITHM SIMPLE-9](https://github.com/manning/CompressionAlgorithms#simple-9) 等。

你应该以多字节（而不是bits）来存储倒排记录表，因为索引的长度和位置都存的是字节信息。

In [106]:
class ECCompressedPostings:
    #If you need any extra helper methods you can add them here 
    ### Begin your code

    ### End your code
    
    @staticmethod
    def encode(postings_list):
        """Encodes `postings_list` 
        
        Parameters
        ----------
        postings_list: List[int]
            The postings list to be encoded
        
        Returns
        -------
        bytes: 
            Bytes reprsentation of the compressed postings list 
        """
        ### Begin your code

        ### End your code

        
    @staticmethod
    def decode(encoded_postings_list):
        """Decodes a byte representation of compressed postings list
        
        Parameters
        ----------
        encoded_postings_list: bytes
            Bytes representation as produced by `CompressedPostings.encode` 
            
        Returns
        -------
        List[int]
            Decoded postings list (each posting is a docId)
        """
        ### Begin your code

        ### End your code

# 作业提交

本次作业用时两周，截止日期为11.4。请大家在截止日期前将代码（包含运行结果，测试内容不作要求），实验报告（可单独撰写，也可整合在jupyter notebook中）一起提交到nkulixuy@163.com，命名方式为`学号_姓名_hw4`。

In [107]:
from math import log
class GammaCompressor:
    @staticmethod
    def int_to_bin(int_in):
        """

        :param int_in:
        :return:
            ret: str

        """
        if int_in == 0:
            return '0'
        # 去除第一个位，因此是print(bin(5)) 0b101
        ret = '1' * int(log(int_in, 2)) + '0' + bin(int_in)[3:]
        return ret

    @staticmethod
    def encode(postings_list):
        """Encodes `postings_list`

        Parameters
        ----------
        postings_list: List[int]
            The postings list to be encoded

        Returns
        -------
        bytes:
            binary reprsentation of the compressed postings list
        """
        ### Begin your code
        encoded_postings_list = ''
        for i in range(0, len(postings_list)):
            #加一是为了处理0和1的情况
            encoded_postings_list += GammaCompressor.int_to_bin(postings_list[i] +1)
        return encoded_postings_list.encode()

    @staticmethod
    def decode(encoded_postings):
        """Decodes a byte representation of compressed postings list

        Parameters
        ----------
        encoded_postings: bytes
            Bytes representation as produced by `CompressedPostings.encode`

        Returns
        -------
        List[int]
            Decoded postings list (each posting is a docId)
        """
        res=[]
        i=0
        byteslen=len(encoded_postings)
        encoded_postings=encoded_postings.decode()
        while True:
            length = 0
            while encoded_postings[i]!='0':
                i+=1
                length+=1
            i+=1
            val= int('1' + encoded_postings[i:i + length],2) - 1
            res.append(val)
            i=i+length
            if i >= byteslen:
                return res


In [108]:
BSBI_instance = BSBIIndex(data_dir='pa1-data',output_dir='gamma_output_dir',postings_encoding=GammaCompressor)
# BSBI_instance.index(False)
for i in range(1, 9):
    with open('dev_queries/query.' + str(i)) as q:
        query = q.read().strip('\n')
        my_results = sorted(set([os.path.normpath(path) for path in BSBI_instance.retrieve(query)]))
        with open('dev_output/' + str(i) + '.out') as o:
            reference_results = sorted(set(['pa1-data\\' + os.path.normpath(x.strip()) for x in o.readlines()]))
            try:
                assert my_results==reference_results
                print("Results match for query:", query.strip())
            except AssertionError:
                print(set(my_results) - set(reference_results))
                print(set(reference_results) - set(my_results))
                print(str(i) + " Results DO NOT match for query: " + query.strip())

Results match for query: we are
Results match for query: stanford class
Results match for query: stanford students
Results match for query: very cool
Results match for query: the
Results match for query: a
Results match for query: the the
Results match for query: stanford computer science
